# Import modules

In [2]:

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


import random
import numpy as np

random.seed(0)
np.random.seed(0)

# user defined methods
import sys
sys.path.append('../utils')

from feature_selection import find_important_features, test_successful_import

# test that module was imported
test_successful_import()

feature_selection imported


upload 'shhs1-dataset-0.20.0.csv' and 'shhs-data-dictionary-0.20.0-variables.csv'

In [3]:
#read Sleep Heart Health Study Dataset

shhs1 = pd.read_csv('../../data/raw/shhs1-dataset-0.20.0.csv', encoding='cp1252', engine='python')
#read Sleep Heart Health Study Dictionary
var_dict = pd.read_csv('../../data/interim/shhs-data-dictionary-0.20.0-variables.csv', encoding='cp1252', engine='python')

In [6]:
#find the shape of shhs1 - 5804 rows, 1271 columns
shhs1.shape

(5804, 1271)

In [7]:
#find the shape of var_dict - 1896 rows, 11 columns
var_dict.shape

(1896, 11)

# Data Processing

In [4]:
# remove sleep monitoring columns from independent variables (unattinable for predicting)
sleep_monitoring_col = var_dict[var_dict['folder'].str.contains(r'sleep monitoring', case=False, na=False)]['id']

In [9]:
sleep_monitoring_col

633        ahiov50
634     headbox_id
635        hrov150
636        hrund30
637     monitor_id
           ...    
1721      timest1p
1722       timest2
1723      timest2p
1724      timest34
1725          waso
Name: id, Length: 1093, dtype: object

In [5]:
# drop target variables, pptidr (has numerical and categorical values in one column), and sleep monitoring columns in independent variable
x = shhs1.drop(columns=['ahi_c0h4a', 'pptidr'])
for col in sleep_monitoring_col:
    if col in x.columns:
        x = x.drop(columns=col)

y = shhs1['ahi_c0h4a']

In [11]:
x

,nsrrid,pptid,ecgdate,lvh3_1,lvh3_3,st4_1_3,st5_1_3,lvhst,mob1,part2deg,...,educat,date02,date25,weight,waist,height,weight20,lang15,age_category_s1,visitnumber
0,200001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-48.0,0.0,65.0,86.0,178.0,69.0,1.0,7,1
1,200002,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,-111.0,-37.0,93.0,107.0,168.0,93.0,NaN,9,1
2,200003,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-7.0,0.0,51.0,82.0,145.0,50.7,1.0,9,1
3,200004,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-24.0,0.0,64.0,85.0,180.0,65.4,1.0,6,1
4,200005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,-89.0,-33.0,56.0,76.0,155.0,56.0,NaN,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,205800,5835,-854.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,-168.0,1.0,70.5,99.0,166.0,70.8,1.0,8,1
5800,205801,5836,-755.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,-95.0,1.0,83.6,99.0,176.0,82.8,1.0,6,1
5801,205802,5837,-768.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,-83.0,1.0,75.0,91.0,175.0,74.2,1.0,7,1
5802,205803,5838,-755.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,-89.0,0.0,76.8,93.0,176.0,78.6,1.0,6,1


In [6]:
# normalize x
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

imputer = KNNImputer(n_neighbors=5)  # You can change the number of neighbors if needed
x_imputed_scaled = imputer.fit_transform(x_scaled)

# reverse scaling
x_imputed = scaler.inverse_transform(x_imputed_scaled)

# Convert the result back to DataFrame
x_imputed = pd.DataFrame(x_imputed, columns=x.columns)

problem with knn: how many neighbors are the best?
NaNs categorical variables will be imputed, but the result might not be intepretable

In [7]:
x_imputed

,nsrrid,pptid,ecgdate,lvh3_1,lvh3_3,st4_1_3,st5_1_3,lvhst,mob1,part2deg,...,educat,date02,date25,weight,waist,height,weight20,lang15,age_category_s1,visitnumber
0,200001.0,1.0,-748.2,0.4,0.6,0.2,2.000000e-01,0.2,0.0,0.0,...,3.0,-48.0,0.0,65.0,86.0,178.0,69.0,1.0,7.0,1.0
1,200002.0,2.0,-75.8,0.2,0.2,0.0,2.000000e-01,0.0,0.0,0.0,...,2.0,-111.0,-37.0,93.0,107.0,168.0,93.0,1.0,9.0,1.0
2,200003.0,3.0,-1071.8,0.2,0.0,0.0,4.000000e-01,0.0,0.0,0.0,...,3.0,-7.0,0.0,51.0,82.0,145.0,50.7,1.0,9.0,1.0
3,200004.0,4.0,-728.6,0.0,0.0,0.0,2.000000e-01,0.0,0.0,0.0,...,3.0,-24.0,0.0,64.0,85.0,180.0,65.4,1.0,6.0,1.0
4,200005.0,5.0,-241.8,0.2,0.0,0.0,2.775558e-17,0.0,0.0,0.0,...,2.0,-89.0,-33.0,56.0,76.0,155.0,56.0,1.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,205800.0,5835.0,-854.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,2.0,-168.0,1.0,70.5,99.0,166.0,70.8,1.0,8.0,1.0
5800,205801.0,5836.0,-755.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,3.0,-95.0,1.0,83.6,99.0,176.0,82.8,1.0,6.0,1.0
5801,205802.0,5837.0,-768.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,4.0,-83.0,1.0,75.0,91.0,175.0,74.2,1.0,7.0,1.0
5802,205803.0,5838.0,-755.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,2.0,-89.0,0.0,76.8,93.0,176.0,78.6,1.0,6.0,1.0


# Feature selection

## Target variable: ahi_c0h4a

### Decision Tree


In [18]:
#create decision tree
tree = DecisionTreeRegressor(random_state = 131)
tree.fit(x_imputed, y)

DecisionTreeRegressor(random_state=131)

In [22]:
find_important_features(x_imputed, shhs1, tree, "decision_tree", "ahi_c0h4a")

/Users/jingwenhu/Documents/GitHub/BCM_CSA_F23/src/notebooks/../utils/feature_selection.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_dataset[target_var] = shhs1[target_var]


254


In [23]:
#identify the most important features through decision trees
tree_importances = tree.feature_importances_
decision_tree_features = np.array(x_imputed.columns)[tree_importances > 0]
decision_tree_dataset = x_imputed[decision_tree_features]
decision_tree_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']

/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77208/122505251.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decision_tree_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']


In [24]:
#end up with 254 features
len(decision_tree_features)

254

In [26]:
#write to csv
decision_tree_dataset.to_csv('../../data/interim/feature_selection/decision_tree_ahi_c0h4a.csv', index=False)

### Random Forest

In [27]:
#create a random forest with 100 "trees"
forest = RandomForestRegressor(n_estimators=100, random_state = 52)  # You can change the number of trees if needed
forest.fit(x_imputed, y)

RandomForestRegressor(random_state=52)

In [28]:
find_important_features(x_imputed, shhs1, forest, "random_forest", "ahi_c0h4a")

/Users/jingwenhu/Documents/GitHub/BCM_CSA_F23/src/notebooks/../utils/feature_selection.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_dataset[target_var] = shhs1[target_var]


279


In [29]:
#identify the most important features through random forest
forest_importances = forest.feature_importances_
random_forest_features = np.array(x_imputed.columns)[forest_importances > 0]
random_forest_dataset = x_imputed[decision_tree_features]
random_forest_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']

/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77208/14452465.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_forest_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']


In [30]:
#end up with 279 features
len(random_forest_features)

279

In [32]:
#write to csv
random_forest_dataset.to_csv('../../data/interim/feature_selection/random_forest_ahi_c0h4a.csv', index=False)

### Mutual Information

In [33]:
#create mutual information model
mi = mutual_info_regression(x_imputed, y, random_state = 568)

In [34]:
# Get feature names where mutual information is greater than 0
mi_features = np.array(x_imputed.columns)[mi > 0]

# Create a new dataset with only the features having non-zero mutual information
mi_dataset = x_imputed[mi_features]

# Add the 'ahi_c0h4a' column to the new dataset
mi_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']

/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77208/3929246669.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mi_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']


In [35]:
#end up with 198 features
len(mi_features)

193

In [36]:
#write to csv
mi_dataset.to_csv('../../data/interim/feature_selection/mi_ahi_c0h4a.csv', index=False)

### Forward selection

In [38]:
#USING AIC
forward_selection_features = []

# Start with an arbitrarily large initial AIC
previous_AIC = float('inf')
AIC_change_threshold = 2  # Adjust this threshold value based on the scale of your AIC

#run forward selection
while len(forward_selection_features) < len(x_imputed.columns):
    remaining_features = list(set(x_imputed.columns) - set(forward_selection_features))
    best_feature_AIC = float('inf')
    best_feature = None

    for feature in remaining_features:
        candidate_features = forward_selection_features.copy()
        candidate_features.append(feature)

        x_forward = x_imputed[candidate_features]
        y_forward = y.copy()

        x_forward = sm.add_constant(x_forward)
        model = sm.OLS(y_forward, x_forward).fit()
        AIC = model.aic

        # Update the best feature if the current one is better (lower AIC)
        if AIC < best_feature_AIC:
            best_feature_AIC = AIC
            best_feature = feature

    # Break the loop if change in AIC is smaller than the threshold or if no significant decrease in AIC
    if abs(best_feature_AIC - previous_AIC) < AIC_change_threshold or best_feature is None:
        print("No significant improvement in AIC or no further significant feature found. Stopping forward selection.")
        break

    forward_selection_features.append(best_feature)
    print(f"Added feature: {best_feature}, AIC with Features: {best_feature_AIC}")

    # Update the previous AIC for the next iteration
    previous_AIC = best_feature_AIC


Added feature: neck20, AIC with Features: 41518.08129720545
Added feature: bmi_s1, AIC with Features: 41364.24158861602
Added feature: age_s1, AIC with Features: 41255.61741102298
Added feature: gender, AIC with Features: 41217.242718044195
Added feature: diasbp, AIC with Features: 41197.10638671003
Added feature: funres02, AIC with Features: 41178.68179985349
Added feature: shhs1_tcvd, AIC with Features: 41168.85552545749
Added feature: ess_s1, AIC with Features: 41159.22469341669
Added feature: nonsp_st, AIC with Features: 41150.241932690806
Added feature: hosnr02, AIC with Features: 41142.70588275835
Added feature: benzod1, AIC with Features: 41134.709576518544
Added feature: ccb1, AIC with Features: 41127.89408107898
Added feature: nsrrid, AIC with Features: 41120.29037287675
Added feature: shhs1_qc, AIC with Features: 41113.16689854111
Added feature: twuweh02, AIC with Features: 41107.11970492668
Added feature: race, AIC with Features: 41100.13620305085
Added feature: estrgn1, AIC

In [39]:
# Create a new dataset with only the features having non-zero mutual information
forward_selection_dataset = x_imputed[forward_selection_features]

# Add the 'ahi_c0h4a' column to the new dataset
forward_selection_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']

#write to csv
forward_selection_dataset.to_csv('../../data/interim/feature_selection/forward_selection_ahi_c0h4a_AIC.csv', index=False)


/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77208/560856859.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forward_selection_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']


In [40]:
#USING BIC
forward_selection_features = []

# Start with an arbitrarily large initial BIC
previous_BIC = float('inf')
BIC_change_threshold = 2  # Adjust this threshold value based on the scale of your BIC

#run forward selection
while len(forward_selection_features) < len(x_imputed.columns):
    remaining_features = list(set(x_imputed.columns) - set(forward_selection_features))
    best_feature_BIC = float('inf')
    best_feature = None

    for feature in remaining_features:
        candidate_features = forward_selection_features.copy()
        candidate_features.append(feature)

        x_forward = x_imputed[candidate_features]
        y_forward = y.copy()

        x_forward = sm.add_constant(x_forward)
        model = sm.OLS(y_forward, x_forward).fit()
        BIC = model.bic

        # Update the best feature if the current one is better (lower BIC)
        if BIC < best_feature_BIC:
            best_feature_BIC = BIC
            best_feature = feature

    # Break the loop if change in BIC is smaller than the threshold or if no significant decrease in BIC
    if abs(best_feature_BIC - previous_BIC) < BIC_change_threshold or best_feature is None:
        print("No significant improvement in BIC or no further significant feature found. Stopping forward selection.")
        break

    forward_selection_features.append(best_feature)
    print(f"Added feature: {best_feature}, Mean Squared Error with Features: {best_feature_BIC}")

    # Update the previous MSE for the next iteration
    previous_BIC = best_feature_BIC


Added feature: neck20, Mean Squared Error with Features: 41531.41390243346
Added feature: bmi_s1, Mean Squared Error with Features: 41384.240496458035
Added feature: age_s1, Mean Squared Error with Features: 41282.282621479
Added feature: gender, Mean Squared Error with Features: 41250.57423111422
Added feature: diasbp, Mean Squared Error with Features: 41237.10420239405
Added feature: funres02, Mean Squared Error with Features: 41225.34591815152
Added feature: shhs1_tcvd, Mean Squared Error with Features: 41222.18594636952
Added feature: ess_s1, Mean Squared Error with Features: 41219.221416942724
Added feature: nonsp_st, Mean Squared Error with Features: 41216.90495883085
No significant improvement in BIC or no further significant feature found. Stopping forward selection.


In [19]:
# Create a new dataset with only the features having non-zero mutual information
forward_selection_dataset = x_imputed[forward_selection_features]

# Add the 'ahi_c0h4a' column to the new dataset
forward_selection_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']

#write to csv
forward_selection_dataset.to_csv('/content/forward_selection_ahi_c0h4a_BIC.csv', index=False)


<ipython-input-19-96fab71f592b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forward_selection_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']


In [ ]:
forward_selection_features = []

# Start with an arbitrarily large initial MSE
previous_mse = float('inf')
mse_change_threshold = 1e-5  # Adjust this threshold value based on the scale of your MSE

#run forward selection
while len(forward_selection_features) < len(x_imputed.columns):
    remaining_features = list(set(x_imputed.columns) - set(forward_selection_features))
    best_feature_mse = float('inf')
    best_feature = None

    for feature in remaining_features:
        candidate_features = forward_selection_features.copy()
        candidate_features.append(feature)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(x_imputed[candidate_features], y, test_size=0.2, random_state=42)

        # Fit a model with the selected features
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Evaluate the model with the added feature
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Update the best feature if the current one is better (lower MSE)
        if mse < best_feature_mse:
            best_feature_mse = mse
            best_feature = feature

    # Break the loop if change in MSE is smaller than the threshold or if no significant decrease in MSE
    if abs(best_feature_mse - previous_mse) < mse_change_threshold or best_feature is None:
        print("No significant improvement in MSE or no further significant feature found. Stopping forward selection.")
        break

    forward_selection_features.append(best_feature)
    print(f"Added feature: {best_feature}, Mean Squared Error with Features: {best_feature_mse}")

    # Update the previous MSE for the next iteration
    previous_mse = best_feature_mse


Added feature: neck20, Mean Squared Error with Features: 79.0476898873102
Added feature: weight, Mean Squared Error with Features: 77.56948755497233
Added feature: age_s1, Mean Squared Error with Features: 76.24158230087114
Added feature: rawvt_s1, Mean Squared Error with Features: 75.7257035698676
Added feature: diasbp, Mean Squared Error with Features: 75.31461213394292
Added feature: asalw15, Mean Squared Error with Features: 75.0504628016291
Added feature: shhs1_tcvd, Mean Squared Error with Features: 74.83050542714896
Added feature: pptid, Mean Squared Error with Features: 74.32050716115323
Added feature: nonsp_st, Mean Squared Error with Features: 74.10989384971398
Added feature: twuweh02, Mean Squared Error with Features: 73.91855108140716
Added feature: ccb1, Mean Squared Error with Features: 73.75153165182127
Added feature: sleepy02, Mean Squared Error with Features: 73.58517596457797
Added feature: hf15, Mean Squared Error with Features: 73.42803420102187
Added feature: smkno

In [ ]:
# Create a new dataset with only the features having non-zero mutual information
forward_selection_dataset = x_imputed[forward_selection_features]

# Add the 'ahi_c0h4a' column to the new dataset
forward_selection_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']

C:\Users\Risto Trajanov\AppData\Local\Temp\ipykernel_54392\2255316604.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forward_selection_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']


In [ ]:
#write to csv
forward_selection_dataset.to_csv('../data/interim/feature_selection/forward_selection_ahi_c0h4a.csv', index=False)

### Backward selection

In [14]:
# USING AIC

# Starting with all the features
backward_selection_features = list(x_imputed.columns)

# Start with an arbitrarily large initial AIC
previous_AIC = float('inf')
AIC_change_threshold = 1.8 # Adjust this threshold value based on the scale of your AIC

#run backward selection
while len(backward_selection_features) > 0:
    removed_feature_AIC = float('inf')
    feature_to_remove = None

    for feature in backward_selection_features:
        candidate_features = backward_selection_features.copy()
        candidate_features.remove(feature)
        
        x_backward = x_imputed[candidate_features]
        y_backward = y.copy()
        
        x_backward = sm.add_constant(x_backward)
        model = sm.OLS(y_backward, x_backward).fit()
        AIC = model.aic
        # Update the best feature to remove if the current one is better (lower MSE)
        if AIC < removed_feature_AIC:
            removed_feature_AIC = AIC
            feature_to_remove = feature

    # Break the loop if change in MSE is smaller than the threshold or if removing a feature increases the MSE
    if abs(removed_feature_AIC - previous_AIC) < AIC_change_threshold or removed_feature_AIC > previous_AIC or best_feature is None:
        print("No significant improvement in AIC or removing any more features deteriorates the model. Stopping backward selection.")
        break

    backward_selection_features.remove(feature_to_remove)
    print(f"Removed feature: {feature_to_remove}, AIC without Feature: {removed_feature_AIC}")

    # Update the previous MSE for the next iteration
    previous_AIC = removed_feature_AIC


41279.28863799625 nsrrid
41279.28863799625 nsrrid
41279.2033718714 ecgdate
41279.2033718714 ecgdate
41279.2033718714 ecgdate
41279.2033718714 ecgdate
41279.2033718714 ecgdate
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19558286416 lvhst
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.19479672044 infmi
41279.194

In [15]:
# Create a new dataset with only the features having non-zero mutual information
backward_selection_dataset = x_imputed[backward_selection_features]

# Add the 'ahi_c0h4a' column to the new dataset
backward_selection_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']

/tmp/ipykernel_4147240/1444284095.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backward_selection_dataset['ahi_c0h4a'] = shhs1['ahi_c0h4a']


In [16]:
#write to csv
backward_selection_dataset.to_csv('../../data/interim/feature_selection/backward_selection_ahi_c0h4a_AIC.csv', index=False)

## Target variable: hf15

In [41]:
y = x_imputed['hf15']
x_imputed = x_imputed.drop(columns=['hf15'])

## Random Forest

In [42]:
forest = RandomForestRegressor(n_estimators=100, random_state = 55)  # You can change the number of trees if needed
forest.fit(x_imputed, y)

RandomForestRegressor(random_state=55)

In [43]:
find_important_features(x_imputed, shhs1, forest, "random_forest", "hf15")

/Users/jingwenhu/Documents/GitHub/BCM_CSA_F23/src/notebooks/../utils/feature_selection.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_dataset[target_var] = shhs1[target_var]


259


In [44]:
forest_importances = forest.feature_importances_
random_forest_features = np.array(x_imputed.columns)[forest_importances > 0]
random_forest_dataset = x_imputed[random_forest_features]
random_forest_dataset['hf15'] = shhs1['hf15']

/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77208/1931634931.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_forest_dataset['hf15'] = shhs1['hf15']


In [45]:
len(random_forest_features)

259

In [46]:
random_forest_dataset.to_csv('../../data/interim/feature_selection/random_forest_hf15.csv', index=False)

### Decision Tree

In [47]:
tree = DecisionTreeRegressor(random_state = 194)
tree.fit(x_imputed, y)

DecisionTreeRegressor(random_state=194)

In [48]:
find_important_features(x_imputed, shhs1, tree, "decision_tree", "hf15")

/Users/jingwenhu/Documents/GitHub/BCM_CSA_F23/src/notebooks/../utils/feature_selection.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_dataset[target_var] = shhs1[target_var]


99


In [49]:
tree_importances = tree.feature_importances_
decision_tree_features = np.array(x_imputed.columns)[tree_importances > 0]
decision_tree_dataset = x_imputed[decision_tree_features]
decision_tree_dataset['hf15'] = shhs1['hf15']

/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77208/1125649490.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decision_tree_dataset['hf15'] = shhs1['hf15']


In [50]:
len(decision_tree_features)

99

In [51]:
decision_tree_dataset.to_csv('../../data/interim/feature_selection/decision_tree_hf15.csv', index=False)

### Mutual Information

In [52]:
mi = mutual_info_regression(x_imputed, y, random_state = 99)

In [53]:
# Get feature names where mutual information is greater than 0
mi_features = np.array(x_imputed.columns)[mi > 0]

# Create a new dataset with only the features having non-zero mutual information
mi_dataset = x_imputed[mi_features]

# Add the 'ahi_c0h4a' column to the new dataset
mi_dataset['hf15'] = shhs1['hf15']

/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77208/1111857069.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mi_dataset['hf15'] = shhs1['hf15']


In [55]:
len(mi_features)

162

In [56]:
mi_dataset.to_csv('../../data/interim/feature_selection/mi_hf15.csv', index=False)

### Forward Selection

In [57]:
forward_selection_features = []

# Start with an arbitrarily large initial MSE
previous_mse = float('inf')
mse_change_threshold = 1e-5  # Adjust this threshold value based on the scale of your MSE

while len(forward_selection_features) < len(x_imputed.columns):
    remaining_features = list(set(x_imputed.columns) - set(forward_selection_features))
    best_feature_mse = float('inf')
    best_feature = None

    for feature in remaining_features:
        candidate_features = forward_selection_features.copy()
        candidate_features.append(feature)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(x_imputed[candidate_features], y, test_size=0.2, random_state=42)

        # Fit a model with the selected features
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Evaluate the model with the added feature
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Update the best feature if the current one is better (lower MSE)
        if mse < best_feature_mse:
            best_feature_mse = mse
            best_feature = feature

    # Break the loop if change in MSE is smaller than the threshold or if no significant decrease in MSE
    if abs(best_feature_mse - previous_mse) < mse_change_threshold or best_feature is None:
        print("No significant improvement in MSE or no further significant feature found. Stopping forward selection.")
        break

    forward_selection_features.append(best_feature)
    print(f"Added feature: {best_feature}, Mean Squared Error with Features: {best_feature_mse}")

    # Update the previous MSE for the next iteration
    previous_mse = best_feature_mse

Added feature: ca15, Mean Squared Error with Features: 0.4269018975236238
Added feature: ntg1, Mean Squared Error with Features: 0.4134223021981537
Added feature: paced, Mean Squared Error with Features: 0.40436839177173456
Added feature: anar1a1, Mean Squared Error with Features: 0.39815950724775123
Added feature: nitro15, Mean Squared Error with Features: 0.3922905564772499
Added feature: afib, Mean Squared Error with Features: 0.38781834254436026
Added feature: emphys15, Mean Squared Error with Features: 0.38446332490938967
Added feature: sob02, Mean Squared Error with Features: 0.38228746166754607
Added feature: mi15, Mean Squared Error with Features: 0.38047594986495437
Added feature: sa15, Mean Squared Error with Features: 0.3790238076926933
Added feature: pacem15, Mean Squared Error with Features: 0.3776377614790318
Added feature: yrsns15, Mean Squared Error with Features: 0.37677148503564145
Added feature: diasbp, Mean Squared Error with Features: 0.37608278109241655
Added feat

In [58]:
len(forward_selection_features)

78

In [59]:
# Create a new dataset with only the features having non-zero mutual information
forward_selection_dataset = x_imputed[forward_selection_features]

# Add the 'ahi_c0h4a' column to the new dataset
forward_selection_dataset['hf15'] = shhs1['hf15']

/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77208/3294433656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forward_selection_dataset['hf15'] = shhs1['hf15']


In [60]:
forward_selection_dataset.to_csv('../../data/interim/feature_selection/forward_selection_hf15.csv', index=False)

### Backward selection

In [8]:
# Starting with all the features
backward_selection_features = list(x_imputed.columns)

# Start with an arbitrarily large initial MSE
previous_mse = float('inf')
mse_change_threshold = 1e-4  # Adjust this threshold value based on the scale of your MSE

while len(backward_selection_features) > 0:
    removed_feature_mse = float('inf')
    feature_to_remove = None

    for feature in backward_selection_features:
        candidate_features = backward_selection_features.copy()
        candidate_features.remove(feature)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(x_imputed[candidate_features], y, test_size=0.2, random_state=42)

        # Fit a model with the selected features
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Evaluate the model without the current feature
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Update the best feature to remove if the current one is better (lower MSE)
        if mse < removed_feature_mse:
            removed_feature_mse = mse
            feature_to_remove = feature

    # Break the loop if change in MSE is smaller than the threshold or if removing a feature increases the MSE
    if abs(removed_feature_mse - previous_mse) < mse_change_threshold or removed_feature_mse > previous_mse:
        print("No significant improvement in MSE or removing any more features deteriorates the model. Stopping backward selection.")
        break

    backward_selection_features.remove(feature_to_remove)
    print(f"Removed feature: {feature_to_remove}, Mean Squared Error without Feature: {removed_feature_mse}")

    # Update the previous MSE for the next iteration
    previous_mse = removed_feature_mse

Removed feature: mh_s1, Mean Squared Error without Feature: 81.01582089575918
Removed feature: o2thpy02, Mean Squared Error without Feature: 78.91039334207044
Removed feature: pgrcar02, Mean Squared Error without Feature: 78.27747138784589
Removed feature: nges02, Mean Squared Error without Feature: 77.75544305465728
Removed feature: srhype, Mean Squared Error without Feature: 77.48295377704235
Removed feature: sa15, Mean Squared Error without Feature: 77.20928421812906
Removed feature: parrptdiab, Mean Squared Error without Feature: 76.96274296116056
Removed feature: weight20, Mean Squared Error without Feature: 76.6807133350899
Removed feature: twuwea02, Mean Squared Error without Feature: 76.40858942856615
Removed feature: noise02, Mean Squared Error without Feature: 76.19360875594528
Removed feature: ca15, Mean Squared Error without Feature: 75.98068522799191
Removed feature: yrssnr02, Mean Squared Error without Feature: 75.80800679717919
Removed feature: thry1, Mean Squared Error 

In [9]:
# Create a new dataset with only the features having non-zero mutual information
backward_selection_dataset = x_imputed[backward_selection_features]

# Add the 'ahi_c0h4a' column to the new dataset
backward_selection_dataset['hf15'] = shhs1['hf15']

/var/folders/sq/7t31xm6962nbdwtkl3xr3kch0000gn/T/ipykernel_77959/3453520002.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backward_selection_dataset['hf15'] = shhs1['hf15']


In [10]:
len(backward_selection_features)

249

In [11]:
backward_selection_dataset.to_csv('../../data/interim/feature_selection/backward_selection_hf15.csv', index=False)